In [47]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'D2Coding'
plt.rcParams['font.size'] = 15
import seaborn as sns
from plotnine import *
import missingno as msno
import folium
import requests

In [42]:
academy = pd.read_csv('./data/동북부학원수.csv')
academy = academy.drop(columns=['Unnamed: 0'], axis=1)
academy

,시군구,법정동,면적(㎢)당 학원 수
0,강북구,미아동,24.907063
1,강북구,번동,10.891089
2,강북구,수유동,15.040650
3,강북구,우이동,3.846154
4,노원구,공릉동,15.337423
5,노원구,상계동,26.041667
6,노원구,월계동,13.235294
7,노원구,중계동,113.450292
8,노원구,하계동,20.731707
9,도봉구,도봉동,4.504505


In [44]:
df = academy.rename(columns={'면적(㎢)당 학원 수': '학원수'})
df

,시군구,법정동,학원수
0,강북구,미아동,24.907063
1,강북구,번동,10.891089
2,강북구,수유동,15.040650
3,강북구,우이동,3.846154
4,노원구,공릉동,15.337423
5,노원구,상계동,26.041667
6,노원구,월계동,13.235294
7,노원구,중계동,113.450292
8,노원구,하계동,20.731707
9,도봉구,도봉동,4.504505


In [45]:
df['법정동']  = df['법정동'].replace('시가지동', '성북동')
df

,시군구,법정동,학원수
0,강북구,미아동,24.907063
1,강북구,번동,10.891089
2,강북구,수유동,15.040650
3,강북구,우이동,3.846154
4,노원구,공릉동,15.337423
5,노원구,상계동,26.041667
6,노원구,월계동,13.235294
7,노원구,중계동,113.450292
8,노원구,하계동,20.731707
9,도봉구,도봉동,4.504505


In [52]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 칼럼을 추가합니다.
df['위도'] = None
df['경도'] = None

# 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in df.iterrows():
    address = row['법정동']
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('addresses'):
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            df.at[index, '위도'] = latitude
            df.at[index, '경도'] = longitude

# 결과를 출력합니다.
df

,시군구,법정동,학원수,위도,경도
0,강북구,미아동,24.907063,37.6270736,127.0269791
1,강북구,번동,10.891089,37.6378913,127.0288473
2,강북구,수유동,15.040650,37.6300865,127.0176014
3,강북구,우이동,3.846154,37.6633,127.002
4,노원구,공릉동,15.337423,37.624874,127.073838
5,노원구,상계동,26.041667,37.6799006,127.0549781
6,노원구,월계동,13.235294,37.6199197,127.0630165
7,노원구,중계동,113.450292,37.6525986,127.0775275
8,노원구,하계동,20.731707,37.6405374,127.0725852
9,도봉구,도봉동,4.504505,37.6786913,127.0434369


In [53]:
df['위도'] = df['위도'].str.replace(',', '').astype(float)
df['경도'] = df['경도'].str.replace(',', '').astype(float)
df

,시군구,법정동,학원수,위도,경도
0,강북구,미아동,24.907063,37.627074,127.026979
1,강북구,번동,10.891089,37.637891,127.028847
2,강북구,수유동,15.040650,37.630086,127.017601
3,강북구,우이동,3.846154,37.663300,127.002000
4,노원구,공릉동,15.337423,37.624874,127.073838
5,노원구,상계동,26.041667,37.679901,127.054978
6,노원구,월계동,13.235294,37.619920,127.063017
7,노원구,중계동,113.450292,37.652599,127.077528
8,노원구,하계동,20.731707,37.640537,127.072585
9,도봉구,도봉동,4.504505,37.678691,127.043437


In [59]:
mean_latitude = data['위도'].mean()
mean_longitude = data['경도'].mean()


AttributeError: 'float' object has no attribute 'mean'

In [57]:
data = df
df_map = folium.Map(location=[data.위도.mean(), data.경도.mean()], zoom_start=12)

for index, df in data.iterrows():
    popup = folium.Popup('{}: {:,.0f}'.format(index, df.학원수), max_width=300)
    folium.CircleMarker(location=[df.위도, df.경도], popup=popup, radius=df.매장수 / 40, 
                        color='red',fill_color='blue').add_to(df_map)

df_map.save('./output/df_map.html')
df_map

AttributeError: 'float' object has no attribute 'mean'

In [8]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,동대문구,신설동,0.22
1,동대문구,용두동,0.76
2,동대문구,제기동,0.72
3,동대문구,전농동,0.94
4,동대문구,답십리동,1.02
5,동대문구,장안동,1.47
6,동대문구,청량리동,0.62
7,동대문구,회기동,0.28
8,동대문구,휘경동,0.86
9,동대문구,이문동,0.91


In [9]:
house = pd.read_csv('./data/동북부집값.csv')
house = house.drop(columns=['Unnamed: 0'], axis=1)
house

,시군구,법정동,면적(㎡)당금액
0,강북구,미아동,990.0
1,강북구,번동,922.0
2,강북구,수유동,965.0
3,강북구,우이동,755.0
4,노원구,공릉동,1016.0
5,노원구,상계동,1115.0
6,노원구,월계동,1213.0
7,노원구,중계동,1169.0
8,노원구,하계동,1222.0
9,도봉구,도봉동,784.0


In [10]:
school = pd.read_csv('./data/동북부학교수.csv')
school = school.drop(columns=['Unnamed: 0'], axis=1)
school

,시군구,법정동,면적(㎢)당 학교 수
0,강북구,미아동,2.230483
1,강북구,수유동,0.406504
2,노원구,공릉동,1.226994
3,노원구,상계동,1.562500
4,노원구,월계동,3.676471
5,노원구,중계동,5.263158
6,노원구,하계동,3.658537
7,도봉구,도봉동,1.801802
8,도봉구,방학동,0.709220
9,도봉구,쌍문동,2.702703
